# Import Necessatry Modules

In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

# Loading WebDriver

In [ ]:
ser = Service('/home/irfan/Downloads/chromedriver_linux64')

# Inititalizing Driver

In [ ]:
driver = Chrome(service=ser)
driver.maximize_window()

## Reditrecting to https://www.alluniversity.info/pakistan/ WebSite

In [ ]:
driver.get('https://www.alluniversity.info/pakistan/')

## Scrape the Name + URL of All Website leading top 10 Universities

In [ ]:
top10 = {}
universities = {}


def loop_over_data(li, d, index=0):
    for link in li:
        anchor = link.find_element(By.TAG_NAME, 'a')
        anc = anchor.get_attribute('href')
        name = link.text[index:]
        if anc not in top10:
            d[name] = anc


def scrape_date():
    try:
        for i in range(3):
            ul = driver.find_elements(By.TAG_NAME, 'ul')
            if i == 0:
                top = ul[0].find_elements(By.TAG_NAME, 'li')
                loop_over_data(top, top10, 3)
                uni = ul[i+1].find_elements(By.TAG_NAME, 'li')
            else:
                uni = ul[0].find_elements(By.TAG_NAME, 'li')
            loop_over_data(uni, universities)
            next_page_link = driver.find_element(By.XPATH, "//a[@title='Next Page']")
            next_page_link.click()
    except NoSuchElementException:
        return

scrape_date()

## Scrape Required Information of Indvidual University

In [ ]:
top = {'Ranking': [], 'Year Established': [], 'Address': [], 'Phone': [], 'Fax': [], 'Website': []}
other = {'Year Established': [], 'Address': [], 'Phone': [], 'Fax': [], 'Website': []}
def loop_over_uni(lst, dict):
    for link in lst:
        driver.get(link)
        tables = driver.find_elements(By.TAG_NAME, 'table')
        for table in tables:
            tr = table.find_elements(By.TAG_NAME, 'tr')
            for row in tr:
                th = row.find_element(By.TAG_NAME, 'th').text
                try:
                    anchor = row.find_element(By.TAG_NAME, 'a')
                    dict[th].append(anchor.get_attribute('href'))
                except:
                    td = row.find_element(By.TAG_NAME, 'td')
                    if len(dict) == 10:
                        span = td.find_element(By.TAG_NAME, 'span')
                        dict[th].append(span.text)
                    else:
                        if th in dict:
                            dict[th].append(td.text)
loop_over_uni(list(top10.values()), top)
loop_over_uni(list(universities.values()), other)